In [10]:
import torch
from tbmalt import Geometry, OrbitalInfo
from tbmalt.physics.dftb import Dftb2
from tbmalt.physics.dftb.feeds import SkFeed, SkfOccupationFeed, HubbardFeed, RepulsiveSplineFeed
from tbmalt.common.maths.interpolation import CubicSpline
from tbmalt.ml.loss_function import Loss, mse_loss

from ase.build import molecule

Tensor = torch.Tensor

# This must be set until typecasting from HDF5 databases has been implemented.
torch.set_default_dtype(torch.float64)

In [11]:
molecule = 'H2O'
target = {'total_energy': torch.tensor([-4.0779])}
shell_dict = {1: [0], 8: [0, 1]}

In [12]:
parameter_db_path = 'mio.h5'

# Type of ML model
model = 'spline'

# Whether performing model fitting
fit_model = True

# Number of training cycles
number_of_epochs = 50

# Learning rate
lr = 0.002

# Loss function
loss_func = mse_loss

In [ ]:
device = torch.device('cpu')

geometry = Geometry(
        torch.tensor([8,1,1], device=device),
        torch.tensor([
            [0.00000000, -0.71603315, -0.00000000],
            [0.00000000, -0.14200298, 0.77844804 ],
            [-0.00000000, -0.14200298, -0.77844804]],
            device=device),units='a')

orbs = OrbitalInfo(geometry.atomic_numbers, shell_dict)

# Identify which species are present
species = torch.unique(geometry.atomic_numbers)
# Strip out padding species and convert to a standard list.
species = species[species != 0].tolist()

In [14]:
h_feed = SkFeed.from_database(parameter_db_path, species, 'hamiltonian', device=device)
s_feed = SkFeed.from_database(parameter_db_path, species, 'overlap', device=device)
o_feed = SkfOccupationFeed.from_database(parameter_db_path, species, device=device)
u_feed = HubbardFeed.from_database(parameter_db_path, species, device=device)
r_feed = RepulsiveSplineFeed.from_database(parameter_db_path, species, device=device)

In [15]:
dftb_calculator = Dftb2(h_feed, s_feed, o_feed, u_feed, r_feed)

ValueError: All `Feeds` must be on the same device